In [6]:
import nbimporter
import rabbits_and_foxes as raf

In [7]:
import numpy as np
import time

# Classes

## class Animal

In [8]:
class Animal:


    def __init__(self, _row, _col, _species, _strategy, 
        _preferred_direction = None, _clockwise = True,
        _diagonal_prediction = False):

        if _species == "rabbit" or _species == "fox":
            self.species = _species
        else:
            raise ValueError("invalid animal species")
        
        self.row = _row
        self.col = _col
        self.strategy = _strategy
        self.preferred_direction = _preferred_direction
        self.clockwise = _clockwise
        self.diagonal_prediction = _diagonal_prediction

        if _species == "rabbit":
            self.animal_index = 0
        elif _species == "fox":
            self.animal_index = 1

    

    def get_position(self):
        return (self.row, self.col)
    
    def get_strategy(self):
        return self.strategy

    def move(self, rabbits_around = None, foxes_around = None):
        if self.strategy == "random":
            dir = raf.random_movement()

        elif self.strategy == "clockwise_escape":
            dir = raf.clockwise_escape(foxes_around,
                self.preferred_direction, self.clockwise)

        elif self.strategy == "chasing_1":
            dir = raf.chasing_1(rabbits_around, self.preferred_direction,
                self.clockwise, self.diagonal_prediction)
        else:
            raise ValueError("invalid strategy")

        # 0 1 2
        # 7   3
        # 6 5 4

        if (dir < 0) or (dir > 7):
            raise ValueError("Animal.move(): invalid direction")

        if dir == 0:
            self.row -= 1
            self.col -= 1
        elif dir == 1:
            self.row -= 1
        elif dir == 2:
            self.row -= 1
            self.col += 1
        elif dir == 3:
            self.col += 1
        elif dir == 4:
            self.row += 1
            self.col += 1
        elif dir == 5:
            self.row += 1
        elif dir == 6:
            self.row += 1
            self.col -= 1
        elif dir == 7:
            self.col -= 1 
    

    def reproduce(self):
        return Animal(self.row, self.col, self.species, self.strategy,
            self.preferred_direction, self.clockwise, self.diagonal_prediction)



In [12]:

rabbit_1 = Animal(1, 3, "rabbit", "random")

In [14]:
rabbit_1.species

'rabbit'

In [13]:
rabbit_1.get_position()


(1, 3)

## Map

In [4]:
class Map:

    def __init__(self, _rows, _cols):
        self.rows = _rows
        self.cols = _cols
        self.population_map = np.zeros((self.rows, self.cols, 2), dtype = int)

        self.objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]
        self.temporal_objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]

        self.hunt_log_map = np.zeros((self.rows, self.cols), dtype = int)

    def set_population_map(self, _population_map):
        self.population_map = _population_map

    def set_objects_map(self, _objects_map):
        self.objects_map = _objects_map

    def get_population_map(self):
        return self.population_map

    def get_objects_map(self):
        return self.objects_map

    def populate(self, amount, species, strategy, 
        preferred_direction = None, clockwise = True,
        diagonal_prediction = False):

        self.population_map = np.zeros((self.rows, self.cols, 2), dtype = int)
        self.objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]
        # self.temporal_objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]
        self.hunt_log_map = np.zeros((self.rows, self.cols), dtype = int)

        if not (species == "rabbit" or species == "fox"):
            raise ValueError("invalid animal species")

        if species == "rabbit":
            animal_index = 0
        elif species == "fox":
            animal_index = 1

        for i in range (amount):
            row = np.random.randint(0, self.rows)
            col = np.random.randint(0, self.cols)
            self.population_map[row, col, animal_index] += 1
            self.objects_map[row, col, animal_index].append(Animal(row, col,
                species, strategy, preferred_direction, clockwise, diagonal_prediction))

    def move_animals(self):

        temporal_objects_map = self.objects_map.copy()
        self.objects_map = [[[[], []] for i in range(self.cols)] for j in range(self.rows)]
        temporal_population_map = self.population_map.copy()
        self.population_map = np.zeros((self.rows, self.cols, 2), dtype = int)

        for i in range(self.rows):
            for j in range(self.cols):
                for animal_index in range(2):
                    for animal in self.objects_map[i, j, animal_index]:
                        rabbits_around = None
                        foxes_around = None
                        if animal.species == "rabbit":
                            if animal_index != 0:
                                raise ValueError("invalid animal index")
                            foxes_around = raf.get_animals_around(temporal_population_map, i, j, animal.species)
                        elif animal.species == "fox":
                            if animal_index != 1:
                                raise ValueError("invalid animal index")
                            rabbits_around = raf.get_animals_around(temporal_population_map, i, j, animal.species)
                        animal.move(rabbits_around, foxes_around)
                        self.objects_map[animal.row, animal.col, animal_index].append(animal)
                        self.population_map[animal.row, animal.col, animal_index] += 1
    
    


    

In [77]:
np.arange(3)

array([0, 1, 2])

# temp

In [20]:
class MyClass:
    classVar = None

    def __init__(self, x, newClassVar = None):
        self.x = x
        if newClassVar is not None:
            MyClass.classVar = newClassVar
    
    def setClassVar(self, newClassVar):
        MyClass.classVar = newClassVar

    def getClassVar(self):
        return MyClass.classVar

    def print_message_then_wait(self, message):
        print(message)
        time.sleep(5)
        print(f"x: {self.x}")

In [32]:
a = MyClass(1)
print(a.getClassVar())

None


In [33]:
b = MyClass(2)
print(b.getClassVar())


None


In [64]:
rows = 2
cols = 3

my_map = [[[[], []] for i in range(cols)] for j in range(rows)]
my_map

[[[[], []], [[], []], [[], []]], [[[], []], [[], []], [[], []]]]

In [74]:
animal_index = 0
my_map[1][0][ animal_index ].append(a)
my_map

[[[[], []], [[], []], [[], []]],
 [[[<__main__.MyClass at 0x7f90ba058590>,
  [[], []],
  [[], []]]]

In [58]:
my_map[1][0].remove(a)
my_map

[[[], [], [], []], [[], [], [], []], [[], [], [], []]]

In [188]:
a = MyClass(1)
print(a.getClassVar())

None


In [203]:
arr2 = [0, 1, 2]
arr2.append(a)
arr2.append(4)
arr2.append(5)
arr2


[0, 1, 2, <__main__.MyClass at 0x7f6664ef92d0>, 4, 5]

In [207]:
arr2.remove(arr2[3])

arr2


IndexError: list index out of range

In [167]:
arr = np.array([1, 2, 3, 4, 5])
arr = np.delete(arr, 1)
print(arr)


[1 3 4 5]


In [159]:
a.print_message_then_wait("hello")

hello
x: 1


In [185]:
arr = np.zeros(6, dtype = object)
arr[0] = a
arr

array([<__main__.MyClass object at 0x7f6664f54190>, 0, 0, 0, 0, 0],
      dtype=object)

In [186]:
arr[4] = MyClass(2)

arr = np.delete(arr, a)
# np.delete(arr, 4)
# arr[4].print_message_then_wait("hello")
# arr = arr[:4]
arr 

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
arr[0].print_message_then_wait("hello")

In [ ]:
print(a.classVar)

In [ ]:
a.setClassVar([[2], 
                [3]])

In [ ]:

print(a.getClassVar())

In [ ]:
b = MyClass(2)


In [ ]:
print(b.getClassVar())

In [ ]:
b.setClassVar([[4],
                [5]])